# AIS vessel prediction
Henrik Månum, student id: 591864

Lars Talian Stangebye-Hansen, student id: 593617

Emil Skogheim, student id: 593632

Kaggle team: "[128] - ;)"

### Kaggle Score
This notebook scored **113.23304** on the public leaderboard on Kaggle, demonstrating strong performance in predicting vessel positions.


Install packages:
```bash
!pip install pandas numpy tqdm scikit-learn xgboost lightgbm matplotlib seaborn searoute shap
```

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from math import radians, cos, sin, asin, sqrt, atan2, degrees
import gc
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from scipy.optimize import minimize
import searoute as sr
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    return distance

def combined_loss_haversine(weights, out_of_fold_predictions, y_true, alpha=0.5):
    weights = np.array(weights)
    if np.isnan(weights).any():
        return np.inf
    weights /= weights.sum()
    combined_preds = np.zeros_like(y_true[['latitude', 'longitude']].values)
    for i, model_name in enumerate(out_of_fold_predictions):
        combined_preds += weights[i] * out_of_fold_predictions[model_name]
    if np.isnan(combined_preds).any():
        return np.inf

    mae = mean_absolute_error(y_true, combined_preds)
    distances = haversine_distance(y_true['latitude'].values, y_true['longitude'].values,
                                   combined_preds[:, 0], combined_preds[:, 1])
    mean_dist = np.mean(distances)

    combined_l = alpha * mae + (1 - alpha) * mean_dist
    return combined_l

In [ ]:
ais_train_path = 'ais_train.csv'
ais_test_path = 'ais_test.csv'
vessels_path = 'vessels.csv'
ports_path = 'ports.csv'
schedules_path = 'schedules_to_may_2024.csv'

try:
    ais_train = pd.read_csv(ais_train_path, delimiter='|')
    ais_test = pd.read_csv(ais_test_path, delimiter=',')
    vessels = pd.read_csv(vessels_path, delimiter='|')
    ports = pd.read_csv(ports_path, delimiter='|')
    schedules = pd.read_csv(schedules_path, delimiter='|')


except Exception as e:
    raise

In [ ]:
columns_to_drop_train = ['portId', 'etaRaw']
train_data = ais_train.drop(columns=columns_to_drop_train)

train_data['time'] = pd.to_datetime(train_data['time'], errors='coerce', utc=True)

train_data = train_data.sort_values(by=['vesselId', 'time']).reset_index(drop=True)

def create_training_data(train_df: pd.DataFrame) -> pd.DataFrame:
    data_rows = []  
    vessel_list = train_df['vesselId'].unique()  
        
    for vessel in tqdm(vessel_list, desc="Featur eenginering"):
        vessel_data = train_df[train_df['vesselId'] == vessel].sort_values(by='time').reset_index(drop=True)
        data_rows_inside = [] 
        last_row = None  
        num_rows = vessel_data.shape[0]
        i = 0

        while num_rows > 0:
            first_time_in_window = vessel_data['time'].min()
            window_end = first_time_in_window + pd.Timedelta(days=5)
            window_end = window_end.replace(hour=23, minute=59, second=59)

            rows_filtered = vessel_data[(vessel_data['time'] >= first_time_in_window) & 
                                        (vessel_data['time'] <= window_end)].copy()
            
            if i == 0 and last_row is None:
                rows_filtered["last_latitude"] = rows_filtered['latitude'].iloc[0]
                rows_filtered["last_longitude"] = rows_filtered["longitude"].iloc[0]
                rows_filtered["last_rot"] = rows_filtered["rot"].iloc[0]
                rows_filtered["last_cog"] = rows_filtered["cog"].iloc[0]
                rows_filtered["last_sog"] = rows_filtered["sog"].iloc[0]
                rows_filtered["last_heading"] = rows_filtered["heading"].iloc[0]
                rows_filtered["last_navstat"] = rows_filtered["navstat"].iloc[0]
                rows_filtered["time_diff"] = 0
                rows_filtered["estimated_distance"] = 0
                rows_filtered["estimated_delta_latitude"] = 0
                rows_filtered["estimated_delta_longitude"] = 0
            else:
                rows_filtered["last_latitude"] = last_row['latitude']
                rows_filtered["last_longitude"] = last_row["longitude"]
                rows_filtered["last_rot"] = last_row["rot"]
                rows_filtered["last_cog"] = last_row["cog"]
                rows_filtered["last_sog"] = last_row["sog"]
                rows_filtered["last_heading"] = last_row["heading"]
                rows_filtered["last_navstat"] = last_row["navstat"]

                rows_filtered["time_diff"] = (rows_filtered["time"] - last_row["time"]).dt.total_seconds() / 3600

                rows_filtered["estimated_distance"] = rows_filtered["last_sog"] * rows_filtered["time_diff"]
                cog_rad = np.deg2rad(rows_filtered["last_cog"])
                earth_radius_nm = 3440.065

                rows_filtered["estimated_delta_latitude"] = (
                    rows_filtered["estimated_distance"] * np.cos(cog_rad) / (earth_radius_nm * (180/np.pi))
                )
                rows_filtered["estimated_delta_longitude"] = (
                    rows_filtered["estimated_distance"] * np.sin(cog_rad) / 
                    (earth_radius_nm * np.cos(np.deg2rad(rows_filtered["last_latitude"])) * (180/np.pi))
                )
    
            last_row = rows_filtered.iloc[-1]
            data_rows_inside.append(rows_filtered)
            vessel_data = vessel_data.drop(rows_filtered.index).reset_index(drop=True)
            num_rows = vessel_data.shape[0]
            i += 1
                        
        vessel_df = pd.concat(data_rows_inside, ignore_index=True)
        data_rows.append(vessel_df)
        
    final_df = pd.concat(data_rows, ignore_index=True)
    return final_df

final_train = create_training_data(train_data)
print(final_train.head())
final_train.to_csv("final_train.csv", index=False)

In [ ]:
ports_data = ports[
    ((ports['latitude'] >= -90) & (ports['latitude'] <= 90)) | ports['latitude'].isnull()
]
ports_data = ports_data[
    ((ports_data['longitude'] >= -180) & (ports_data['longitude'] <= 180)) | ports_data['longitude'].isnull()
]

columns_to_drop_ports = ['UN_LOCODE', 'ISO', 'countryName', 'portLocation']
ports_data = ports_data.drop(columns=columns_to_drop_ports)

cleaned_ports_file_path = "cleaned_ports_dataset.csv"
ports_data.to_csv(cleaned_ports_file_path, index=False)

ports_data_refined = pd.read_csv(cleaned_ports_file_path)

from sklearn.neighbors import BallTree

ports_coords = ports_data_refined[['latitude', 'longitude']].dropna().values
ports_coords_rad = np.radians(ports_coords)

port_tree = BallTree(ports_coords_rad, metric='haversine')

def distance_to_nearest_port(lat, lon):
    try:
        point_rad = np.radians([[lat, lon]])

        dist, idx = port_tree.query(point_rad, k=1)

        distance_km = dist[0][0] * 6371.0
        return distance_km
    except Exception as e:
        print("Error:", e)
        return np.nan

In [ ]:
def create_interaction_features(df: pd.DataFrame) -> pd.DataFrame:
    df['sog_heading_interaction'] = df['last_sog'] * df['last_heading']
    df['cog_rot_interaction'] = df['last_cog'] * df['last_rot']
    df['time_diff_distance_interaction'] = df['time_diff'] * df['estimated_distance']
    
    return df

def create_time_features(df: pd.DataFrame) -> pd.DataFrame:
    df['hour_of_day'] = df['time'].dt.hour                 
    df['day_of_week'] = df['time'].dt.dayofweek            
    df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)  
    
    return df


In [ ]:
test_data = ais_test
test_data['time'] = pd.to_datetime(test_data['time'], errors='coerce', utc=True)

test_data = test_data.sort_values(by=['vesselId', 'time']).reset_index(drop=True)

def create_test_features(train_df: pd.DataFrame, test_df: pd.DataFrame) -> pd.DataFrame:
    data_rows = []  
    vessel_list = test_df['vesselId'].unique() 

    for vessel in tqdm(vessel_list, desc="Feature Engineering on Test Data"):
        test_vessel_data = test_df[test_df['vesselId'] == vessel].sort_values(by='time').reset_index(drop=True)
        
        last_train_rows = train_df[train_df['vesselId'] == vessel]
        vessel_last_row = last_train_rows.iloc[-1] if not last_train_rows.empty else None
        
        if vessel_last_row is not None:
            test_vessel_data["last_latitude"] = vessel_last_row['latitude']
            test_vessel_data["last_longitude"] = vessel_last_row["longitude"]
            test_vessel_data["last_sog"] = vessel_last_row["sog"]
            test_vessel_data["last_cog"] = vessel_last_row["cog"]
            test_vessel_data["last_heading"] = vessel_last_row["heading"]
            test_vessel_data["last_rot"] = vessel_last_row["rot"]
            test_vessel_data["last_navstat"] = vessel_last_row["navstat"]
            
            test_vessel_data["time_diff"] = (test_vessel_data["time"] - vessel_last_row["time"]).dt.total_seconds() / 3600
            
            test_vessel_data["estimated_distance"] = test_vessel_data["last_sog"] * test_vessel_data["time_diff"]
            cog_rad = np.deg2rad(test_vessel_data["last_cog"]) 
            earth_radius_nm = 3440.065 
            test_vessel_data["estimated_delta_longitude"] = (
                test_vessel_data["estimated_distance"] * np.sin(cog_rad) / 
                (earth_radius_nm * np.cos(np.deg2rad(test_vessel_data["last_latitude"])) * (180 / np.pi))
            )

            test_vessel_data["estimated_delta_latitude"] = (
                test_vessel_data["estimated_distance"] * np.cos(cog_rad) / (earth_radius_nm * (180 / np.pi))
            )
        else:
            test_vessel_data["last_latitude"] = np.nan
            test_vessel_data["last_longitude"] = np.nan
            test_vessel_data["last_cog"] = np.nan
            test_vessel_data["last_rot"] = np.nan
            test_vessel_data["last_navstat"] = np.nan
            test_vessel_data["last_heading"] = np.nan
            test_vessel_data["last_sog"] = np.nan
            test_vessel_data["time_diff"] = 0
            test_vessel_data["estimated_delta_latitude"] = 0
            test_vessel_data["estimated_delta_longitude"] = 0
            test_vessel_data["estimated_distance"] = 0

        data_rows.append(test_vessel_data)
    
    final_test = pd.concat(data_rows, ignore_index=True)

    final_test = final_test.sort_values(by="ID").reset_index(drop=True)

    return final_test  

final_test = create_test_features(final_train, test_data)

final_test.to_csv("final_test.csv", index=False)

In [ ]:
final_train['distance_to_nearest_port'] = final_train.apply(
    lambda row: distance_to_nearest_port(row['latitude'], row['longitude']),
    axis=1
)

final_test['distance_to_nearest_port'] = final_test.apply(
    lambda row: distance_to_nearest_port(row['last_latitude'], row['last_longitude']),
    axis=1
)

final_train = create_interaction_features(final_train)
final_test = create_interaction_features(final_test)

final_train = create_time_features(final_train)
final_test = create_time_features(final_test)

In [ ]:
final_train['time'] = pd.to_datetime(final_train['time'], errors='coerce', utc=True)
final_test['time'] = pd.to_datetime(final_test['time'], errors='coerce', utc=True)

feat_cols = [
    'last_latitude', 'last_longitude', 'last_rot', 'last_cog', 'last_sog', 'last_heading', 'last_navstat',
    'time_diff', 'estimated_distance', 'estimated_delta_latitude', 'estimated_delta_longitude',
    'distance_to_nearest_port', 'sog_heading_interaction', 'cog_rot_interaction', 'time_diff_distance_interaction',
    'hour_of_day', 'day_of_week', 'is_weekend'
]
target_columns = ['latitude', 'longitude']

train_missing_feat = set(feat_cols) - set(final_train.columns)
test_missing_feat = set(feat_cols) - set(final_test.columns)

if train_missing_feat:
    raise ValueError(f"Missing feature columns in training data: {train_missing_feat}")

if test_missing_feat:
    raise ValueError(f"Missing feature columns in test data: {test_missing_feat}")

X_train = final_train[feat_cols]
y_train = final_train[target_columns]
X_test = final_test[feat_cols]


In [22]:
models = {
    'LightGBM': MultiOutputRegressor(lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)),
    'RandomForest': MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42, max_depth=6)),
    'XGBoost': MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, seed=42)),
}

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

out_of_fold_predictions = {model_name: np.zeros((X_train.shape[0], y_train.shape[1])) for model_name in models}
test_preds = {model_name: np.zeros((X_test.shape[0], y_train.shape[1], n_splits)) for model_name in models}


for fold, (train_idx, valid_idx) in enumerate(kf.split(X_train, y_train)):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

    for model_name, model in models.items():
        
        model.fit(X_tr, y_tr)

        y_pred_valid = model.predict(X_val)
        y_pred_test = model.predict(X_test)

        out_of_fold_predictions[model_name][valid_idx] = y_pred_valid
        test_preds[model_name][:, :, fold] = y_pred_test



KeyboardInterrupt: 

In [ ]:
methods = ['L-BFGS-B', 'SLSQP', 'TNC']

init_w = np.ones(len(models)) / len(models)

constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1} 
bounds = [(0, 1)] * len(models) 

loss_best = np.inf
w_best = None
method_best = None
alpha = 0.5

for method in methods:
    try:
        res = minimize(
            combined_loss_haversine,
            init_w,
            args=(out_of_fold_predictions, y_train, alpha),
            bounds=bounds,
            method=method,
            constraints=constraints
        )

        if res.fun < loss_best and res.success:
            loss_best = res.fun
            w_best = res.x / res.x.sum() 
            method_best = method

    except Exception as e:
        print(f"Error with {method}: {e}")

if w_best is not None and method_best is not None:
    pass
else:
    w_best = init_w

print(f"Best optimization method: {method_best}")
print("Best weights:")
for i, model_name in enumerate(models):
    print(f"{model_name}: {w_best[i]:.4f}")

for model_name in models:
    test_preds[model_name] = np.mean(test_preds[model_name], axis=2)


In [ ]:
final_test_pred = np.zeros((X_test.shape[0], y_train.shape[1]))
for i, model_name in enumerate(models):
    final_test_pred += w_best[i] * test_preds[model_name]

prediction_df = pd.DataFrame({
    'ID': final_test['ID'],
    'longitude_predicted': final_test_pred[:, 1],
    'latitude_predicted': final_test_pred[:, 0]
})

prediction_output_filename = 'predictions123COMPERE.csv'
prediction_df.to_csv(prediction_output_filename, index=False)

prediction_df['ID'] = prediction_df['ID'].astype(int)

prediction_df = prediction_df.sort_values(by='ID').reset_index(drop=True)

sorted_prediction_output_filename = 'predictionsCOMPARE.csv'
prediction_df.to_csv(sorted_prediction_output_filename, index=False)